In [1]:
import re

from bs4 import BeautifulSoup

## Загрузка HTML-страниц и извлечение предложений

In [2]:
from itertools import filterfalse

def extract_text(filename):
    from bs4 import Comment
    
    with open(filename, mode='r') as f_name:
        soup = BeautifulSoup(f_name, 'html.parser')
    
    for a in soup.find_all('a', href=True):
        a.replaceWithChildren()
    
    tags = soup.find_all(text=True)
    tags = filter(lambda t: t.parent.name not in {'script', 'style', 'head'}, tags)
    tags = filterfalse(lambda t: isinstance(t, Comment), tags)    
    tags = map(lambda t: t.replace('\xa0', ' '), tags)
    
    return ' '.join(tags)

In [3]:
SPACES_PATTERN = re.compile("\s+")
TEXT_PATTERN = re.compile("\w+")

def extract_sentences(text):
    splits = [0]
    
    braces = 0
    for i, c in enumerate(text):
        if c in '([{':
            braces += 1
        elif c in ')]}' and braces > 0:
            braces -= 1
        elif c == '.' and braces == 0:
            if i >= 3 and ' ' not in text[i-3:i-1]:
                splits.append(i + 1)
    splits.append(len(text))
    
    sentences = (text[i:j].strip() for i, j in zip(splits[:-1], splits[1:]))
    sentences = filter(None, map(lambda s: SPACES_PATTERN.sub(" ", s), sentences))
    sentences = filter(lambda s: TEXT_PATTERN.search(s), sentences)
    return list(sentences)

In [4]:
from itertools import chain

SPACES_REMOVE_AFTER_PATTERN  = re.compile("\s+(?=[.,;s?!\}\)\]»\"\'])")
SPACES_REMOVE_BEFORE_PATTERN = re.compile("(?<=[\{\(\[«\"\'])\s+")

def extract_sentences_from_text(text):
    text = SPACES_REMOVE_AFTER_PATTERN.sub("", text)
    text = SPACES_REMOVE_BEFORE_PATTERN.sub("", text)
    text = text.split('\n')
    text = filter(None, map(lambda s: s.strip(), text))
    text = chain(*map(extract_sentences, text))
    text = filter(lambda s: s.count(' ') > 3, text)
    return list(text)

In [5]:
import os

docs = []

for dirpath, dirnames, filenames in os.walk("dumps"):
    filenames = map(lambda f: os.path.join(dirpath, f), filenames)
    filenames = list(filenames)
    
for filename in filenames:
    text = extract_text(filename)
    sentences = extract_sentences_from_text(text)
    docs.extend(sentences)

In [6]:
len(docs)

2866

## Морфологическая предобработка предложений

In [7]:
import re

from functools import lru_cache

from pymorphy2 import MorphAnalyzer
from nltk.stem import WordNetLemmatizer

ru_morph = MorphAnalyzer()
en_morph = WordNetLemmatizer()

CYRILLIC_PATTERN = re.compile('[а-яА-Я]')
DIGIT_PATTERN = re.compile('\d+')

def has_cyrillic(text):
    return bool(CYRILLIC_PATTERN.search(text))

def has_numeric(text):
    return bool(DIGIT_PATTERN.search(text))

@lru_cache(maxsize=1500)
def morph_process(token):
    if has_cyrillic(token):
        return ru_morph.parse(token)[0].normal_form
    elif has_numeric(token):
        return token
    else:
        return en_morph.lemmatize(token)
    
def morph_sentence(sentence):
    res = re.findall('\d+', sentence.lower())
    words = re.findall('[^\W\d_]+', sentence.lower())
    res.extend(map(morph_process, words))
    return res

## Текстовое ранжирование

In [8]:
# NOT IMPLEMENTED

from nltk.corpus import stopwords

ru_stopwords = stopwords.words('russian')
en_stopwords = stopwords.words('english')

stopwords_ = set(ru_stopwords) | set(en_stopwords)

### TF-IDF 

In [9]:
import numpy as np
from scipy.sparse import csr_matrix


class TfIdf:
    def __init__(self, preprocess, stopwords=None, idf=True, norm=True):
        self.preprocess_ = preprocess
        self.vocabulary_ = {}
        
        self._use_idf = idf
        self._use_norm = norm
        
    def _transform(self, docs, use_vocab=True):
        if not use_vocab:
            self.vocabulary_ = {}
        vocabulary = self.vocabulary_
        
        df_ = {}
        tf_, indices, indptr = [], [], [0]
        
        for doc_id, doc in enumerate(docs):
            tf_counts, df_entries = {}, set()
            
            for term in self.preprocess_(doc):                
                if term not in vocabulary:
                    if use_vocab:
                        continue
                    vocabulary[term] = len(vocabulary)
                term_id = vocabulary[term]
                
                if term_id not in tf_counts:
                    tf_counts[term_id] = 0
                tf_counts[term_id] += 1
                
                df_entries.add(term_id)
                
            indices.extend(tf_counts.keys())
            tf_.extend(tf_counts.values())
            indptr.append(len(indices))
            
            for term_id in df_entries:
                df_[term_id] = df_.get(term_id, 0) + 1
                
        tf_ = csr_matrix((tf_, indices, indptr), dtype=np.uint32,
                         shape=(len(docs), len(vocabulary)))
        tf_.sort_indices()
        tf_ = tf_.toarray()
        
        if self._use_norm:
            # norm = np.sqrt(np.power(tf_, 2).sum(axis=1, keepdims=True))
            norm = tf_.max(axis=1, keepdims=True)
            tf_ = tf_ / norm
        
        if not use_vocab:
            self.idf_ = [np.log10(len(docs) / float(df_[tid])) for tid in range(len(vocabulary))]
            self.idf_ = np.asarray(self.idf_)
        idf_ = self.idf_
        
        if self._use_idf:
            tf_ = tf_ * idf_.reshape(1, -1)
        
        return tf_
        
    def fit_transform(self, docs):
        return self._transform(docs, use_vocab=False)
    
    def fit(self, docs):
        _ = self.fit_transform(docs)
        return self
    
    def transform(self, docs):
        return self._transform(docs, use_vocab=True)

### TF-IDF Ranker

In [10]:
from sklearn.metrics.pairwise import cosine_similarity

In [11]:
class TfIdfRanker:
    def __init__(self, tfidf):
        self._ranker_tfidf = tfidf
    
    def load_docs(self, docs):
        self.forward_index = docs
        self.X_docs = self._ranker_tfidf.fit_transform(docs)
        
    def rank(self, *queries):
        X_docs  = self.X_docs
        X_query = self._ranker_tfidf.transform(*queries)
        
        ranks = cosine_similarity(X_query, X_docs)
        a_ranks = np.argsort(ranks, axis=1)[:,::-1]
        return [[(doc_id, ranks[qid, doc_id]) for doc_id in a_ranks[qid, :]]
                for qid in range(a_ranks.shape[0])]
    
    def print_serp(self, query, relevant_docs, top=10, norm=False):
        print("QUERY:", query)
        if norm:
            normed = self._ranker_tfidf.preprocess_(query)
            print("[NORMALIZED]")
            print(*normed)
        print("\n")
            
        ranks = self.rank([query])[0]
        
        for pos, (doc_id, rank) in enumerate(ranks[:top], 1):
            sent = self.forward_index[doc_id]
            mark = "[{}]".format('*' if doc_id in relevant_docs else 'o')
            
            print(f"POS:   {pos}")
            print(f"DOCID: {doc_id}")
            print(f"RANK:  {rank:.10f}")
            print()
            print(mark, sent)
            
            if norm:
                normed = self._ranker_tfidf.preprocess_(sent)
                print()
                print("[NORMALIZED]")
                print(*normed)
            
            print("=" * 80)
            
        return ranks

### Language Model Ranker

In [12]:
class LangModelRanker:
    def __init__(self, lambda_=0.7):
        self._ranker_tfidf = TfIdf(preprocess=morph_sentence, idf=False)
        self.lambda_ = lambda_
    
    def load_docs(self, docs):
        self.forward_index = docs
        
        X_docs = self._ranker_tfidf.fit_transform(docs)
        
        # условная вероятность P(term|doc)
        self.p2doc = X_docs / X_docs.sum(axis=1).reshape(-1, 1)
        
        # условная вероятность P(term|model)
        self.p2model = X_docs.sum(axis=0)
        self.p2model = self.p2model / self.p2model.sum()
        
        self._probs = (1.0 - self.lambda_) * self.p2model + self.lambda_ * self.p2doc
        
    def rank(self, queries):        
        X_query = self._ranker_tfidf.transform(queries)
        
        ranks = []
        for qid in range(len(queries)):
            i_query = np.nonzero(X_query[qid])[0]
            rank = self._probs[:, i_query].prod(axis=1)
            ranks.append(rank)
        ranks = np.asarray(ranks)
            
        a_ranks = np.argsort(ranks, axis=1)[:,::-1]
        return [[(doc_id, ranks[qid, doc_id]) for doc_id in a_ranks[qid, :]]
                for qid in range(a_ranks.shape[0])]
    
    def print_serp(self, query, relevant_docs, top=10, norm=False, probs=False):
        print("QUERY:", query)
        if norm:
            normed = self._ranker_tfidf.preprocess_(query)
            print("[NORMALIZED]")
            print(*normed)
        print()
        
        if probs:
            X_query = self._ranker_tfidf.transform([query])
            i_query = np.nonzero(X_query[0])[0]
            v_inverse = {v: k for k, v in self._ranker_tfidf.vocabulary_.items()}
            for i in i_query:
                print("P({term}|model) = {prob:.5f}".format(term=v_inverse[i], prob=self.p2model[i]))
            print()
            
        ranks = self.rank([query])[0]
        
        for pos, (doc_id, rank) in enumerate(ranks[:top], 1):
            sent = self.forward_index[doc_id]
            mark = "[{}]".format('*' if doc_id in relevant_docs else 'o')
            
            print(f"POS:   {pos}")
            print(f"DOCID: {doc_id}")
            print(f"RANK:  {rank}")
            print()
            print(mark, sent)
            
            if norm:
                normed = self._ranker_tfidf.preprocess_(sent)
                print()
                print("[NORMALIZED]")
                print(*normed)
            
            print("=" * 80)
            
        return ranks

### Метрики

In [13]:
from operator import itemgetter


def DCG_score(ranks_true, ranks_pred):
    score = 0
    for pos, (doc_id, rank) in enumerate(ranks_pred, 1):
        score += ranks_true.get(doc_id, 0) / np.log2(pos + 1)
    return score

def NDCG_score(ranks_true, ranks_pred):
    ranks_true_ = sorted(ranks_true.items(), key=itemgetter(1), reverse=True)
    
    dcg_curr = DCG_score(ranks_true, ranks_pred)
    dcg_best = DCG_score(ranks_true, ranks_true_)
    
    return dcg_curr / dcg_best

In [14]:
def count_metrics(ranks_true, ranks_pred):
    TP, TN, FP, FN = 0.0, 0.0, 0.0, 0.0
    
    docs_processed = set()
    
    for doc_id, rank in ranks_pred:
        y_pred = rank
        y_true = ranks_true.get(doc_id, 0) > 0

        TP += y_true and y_pred
        TN += not y_true and not y_pred
        FP += not y_true and y_pred
        FN += y_true and not y_pred
        
        docs_processed.add(doc_id)        

    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    fmeasure = 2.0 / (1.0 / recall + 1.0 / precision)
    ndcg_10 = NDCG_score(ranks_true, ranks_pred[:10])
    
    ndcg = NDCG_score(ranks_true, ranks_pred)
    
    return precision, recall, fmeasure, ndcg_10, ndcg

### Запросы

In [15]:
queries = [
    "Лётчику понадобилось два десятка лет, чтобы повторно стать подполковником.",
    "Не вызвавший первоначально интереса экспертов «маленький рыжеволосый мальчик» " + \
        "(на илл.) выставлен на аукционе со стартовой ценой в миллион евро.",
    "Американский фотограф, известный съёмками высшего общества, был также личным " + \
        "фотографом знаменитого мафиозо.",
]

In [16]:
relevant_docs = [
    {146:  2, 148:  2, 149:  2, 165:  2, 180:  2},
    {2375: 1, 2389: 2, 2390: 2, 2392: 1, 2416: 2},
    {1161: 1, 2557: 2, 2581: 2, 2582: 2},
]

In [17]:
qid = 0

for doc_id in relevant_docs[qid]:
    print(f"docid#{doc_id}")
    print(docs[doc_id], '\n')

docid#146
подполковник Российской императорской армии полковник Красной армии 

docid#148
Николай Александрович Мульков (15 [27] ноября 1890 — не ранее 1952) — военный лётчик. 

docid#149
Подполковник Российской императорской армии (1917), участник Первой мировой войны. 

docid#165
19 мая 1916 года Николай Мульков был произведён в чин штабс-капитана, 17 августа 1917 года в капитаны, а на следующий день — в подполковники. 

docid#180
11 сентября 1940 года получил звание подполковника, а 21 февраля 1945 года — полковника. 



In [18]:
qid = 1

for doc_id in relevant_docs[qid]:
    print(f"docid#{doc_id}")
    print(docs[doc_id], '\n')

docid#2375
«Ребёнок Иисус, стоящий на коленях перед орудиями Страстей» (фр. «Jésus enfant agenouillé devant les instruments de la Passion») — картина, находящаяся в частной коллекции в Вандее во Франции и атрибутируемая братьям Ленен или, по мнению некоторых искусствоведов, Матьё Ленену [1]. 

docid#2389
Владелица увидела Руйяков по телевидению; в разговоре по телефону она называла картину «Маленький рыжеволосый мальчик». 

docid#2390
Она сообщила, что ряд парижских аукционистов, к которым она уже обращалась некоторое время назад, не заинтересовались картиной, ссылаясь на слабый интерес рынка к картинам на религиозный сюжет [6]. 

docid#2392
Он увидел картину висящей на стене, сразу определил её сюжет и оценил выдающиеся художественные достоинства [7]. 

docid#2416
Она оценивается от 3 до 5 млн евро (по другим данным, только в 1 млн евро [2]) — в три раза больше, чем картина «Святой Иероним», которая должна быть выставлена в апреле 2018 года на аукционе «Кристис» (1 млн долларов). 



In [19]:
qid = 2

for doc_id in relevant_docs[qid]:
    print(f"docid#{doc_id}")
    print(docs[doc_id], '\n')

docid#1161
Лаки Лучано был любимой моделью американского фотографа Слима Ааронса. 

docid#2557
Джордж Аллен «Слим» Ааронс (29 октября 1916 — 30 мая 2006) — американский фотограф, известный съемками «красивой жизни» высшего общества, светских персон, кинозвёзд и прочих знаменитостей. 

docid#2581
Из общего ряда «гламурных» персонажей, которых фотографировал Слим Ааронс, резко выделялся гангстер Чарльз Лучано по прозвищу «Lucky» («Счастливчик») — один из «боссов» итало-американской организованной преступности. 

docid#2582
Ааронс следовал за ним повсюду, в том числе в Италию, и сделал множество снимков «крестного отца». 



### Модель 1: нормализация без idf

In [20]:
scores = {}

In [21]:
tf_idf = TfIdf(preprocess=morph_sentence, idf=False)

ranker = TfIdfRanker(tf_idf)
ranker.load_docs(docs)

In [22]:
qid = 0

ranks = ranker.print_serp(queries[qid], relevant_docs[qid], norm=True)
ranks = [(doc_id, int(i < 10)) for i, (doc_id, _) in enumerate(ranks)]
metrics = count_metrics(relevant_docs[qid], ranks)

print()
print(("Precision: {:>.10f}\n" + \
       "Recall:    {:>.10f}\n" + \
       "F-measure: {:>.10f}\n" + \
       "NDCG-10:   {:>.10f}").format(*metrics))

scores[(qid, False)] = metrics

QUERY: Лётчику понадобилось два десятка лет, чтобы повторно стать подполковником.
[NORMALIZED]
лётчик понадобиться два десятка год чтобы повторно стать подполковник


POS:   1
DOCID: 1079
RANK:  0.2738612788

[o] С 2001 года работает в Гарвардском университете, спустя три года стал его профессором.

[NORMALIZED]
2001 с год работать в гарвардский университет спустя три год стать он профессор
POS:   2
DOCID: 180
RANK:  0.2738612788

[*] 11 сентября 1940 года получил звание подполковника, а 21 февраля 1945 года — полковника.

[NORMALIZED]
11 1940 21 1945 сентябрь год получить звание подполковник а февраль год полковник
POS:   3
DOCID: 398
RANK:  0.2500000000

[o] В 1888 году весь Бруней стал британским протекторатом.

[NORMALIZED]
1888 в год весь бруней стать британский протекторат
POS:   4
DOCID: 528
RANK:  0.2433321317

[o] С 1993 г. с периодичностью раз в два года проводятся национальные спартакиады (Pesta Sukan Kebangsaan).

[NORMALIZED]
1993 с год с периодичность раз в два год провод

In [23]:
qid = 1

ranks = ranker.print_serp(queries[qid], relevant_docs[qid], norm=True)
ranks = [(doc_id, int(i < 10)) for i, (doc_id, _) in enumerate(ranks)]
metrics = count_metrics(relevant_docs[qid], ranks)

print()
print(("Precision: {:>.10f}\n" + \
       "Recall:    {:>.10f}\n" + \
       "F-measure: {:>.10f}\n" + \
       "NDCG-10:   {:>.10f}").format(*metrics))

scores[(qid, False)] = metrics

QUERY: Не вызвавший первоначально интереса экспертов «маленький рыжеволосый мальчик» (на илл.) выставлен на аукционе со стартовой ценой в миллион евро.
[NORMALIZED]
не вызвать первоначально интерес эксперт маленький рыжеволосый мальчик на илл выставить на аукцион с стартовый цена в миллион евро


POS:   1
DOCID: 2416
RANK:  0.3712129349

[*] Она оценивается от 3 до 5 млн евро (по другим данным, только в 1 млн евро [2]) — в три раза больше, чем картина «Святой Иероним», которая должна быть выставлена в апреле 2018 года на аукционе «Кристис» (1 млн долларов).

[NORMALIZED]
3 5 1 2 2018 1 она оцениваться от до миллион евро по другой данные только в миллион евро в три раз большой чем картина святой иероним который должный быть выставить в апрель год на аукцион кристис миллион доллар
POS:   2
DOCID: 27
RANK:  0.3211820274

[o] Несмотря на то, что он не входил в коллегию кардиналов, в 1958 году он получил на конклаве несколько голосов.

[NORMALIZED]
1958 несмотря на то что он не входить в ко

In [24]:
qid = 2

ranks = ranker.print_serp(queries[qid], relevant_docs[qid], norm=True)
ranks = [(doc_id, int(i < 10)) for i, (doc_id, _) in enumerate(ranks)]
metrics = count_metrics(relevant_docs[qid], ranks)

print()
print(("Precision: {:>.10f}\n" + \
       "Recall:    {:>.10f}\n" + \
       "F-measure: {:>.10f}\n" + \
       "NDCG-10:   {:>.10f}").format(*metrics))

scores[(qid, False)] = metrics

QUERY: Американский фотограф, известный съёмками высшего общества, был также личным фотографом знаменитого мафиозо.
[NORMALIZED]
американский фотограф известный съёмка высокий общество быть также личный фотограф знаменитый мафиозо


POS:   1
DOCID: 2557
RANK:  0.3962969620

[*] Джордж Аллен «Слим» Ааронс (29 октября 1916 — 30 мая 2006) — американский фотограф, известный съемками «красивой жизни» высшего общества, светских персон, кинозвёзд и прочих знаменитостей.

[NORMALIZED]
29 1916 30 2006 джордж аллен слить ааронс октябрь май американский фотограф известный съёмка красивый жизнь высокий общество светский персона кинозвезда и прочий знаменитость
POS:   2
DOCID: 1161
RANK:  0.3698001308

[*] Лаки Лучано был любимой моделью американского фотографа Слима Ааронса.

[NORMALIZED]
лак лучано быть любимый модель американский фотограф слима ааронс
POS:   3
DOCID: 48
RANK:  0.2264554068

[o] Фотография Павла VI, фотограф Лотар Воллех

[NORMALIZED]
фотография павел vi фотограф лотар воллех
POS

### Модель 2: нормализация c idf

In [25]:
tf_idf = TfIdf(preprocess=morph_sentence, idf=True)

ranker = TfIdfRanker(tf_idf)
ranker.load_docs(docs)

In [26]:
qid = 0

ranks = ranker.print_serp(queries[qid], relevant_docs[qid], norm=True)
ranks = [(doc_id, int(i < 10)) for i, (doc_id, _) in enumerate(ranks)]
metrics = count_metrics(relevant_docs[qid], ranks)

print()
print(("Precision: {:>.10f}\n" + \
       "Recall:    {:>.10f}\n" + \
       "F-measure: {:>.10f}\n" + \
       "NDCG-10:   {:>.10f}").format(*metrics))

scores[(qid, True)] = metrics

QUERY: Лётчику понадобилось два десятка лет, чтобы повторно стать подполковником.
[NORMALIZED]
лётчик понадобиться два десятка год чтобы повторно стать подполковник


POS:   1
DOCID: 2190
RANK:  0.2214248002

[o] Он был вынужден повторно учиться в 10 классе.

[NORMALIZED]
10 он быть вынужденный повторно учиться в класс
POS:   2
DOCID: 2213
RANK:  0.1762893825

[o] Полицейские повторно допросили Скотта по телефону, но он отрицал причастность.

[NORMALIZED]
полицейский повторно допросить скотт по телефон но он отрицать причастность
POS:   3
DOCID: 180
RANK:  0.1620625084

[*] 11 сентября 1940 года получил звание подполковника, а 21 февраля 1945 года — полковника.

[NORMALIZED]
11 1940 21 1945 сентябрь год получить звание подполковник а февраль год полковник
POS:   4
DOCID: 149
RANK:  0.1495289028

[*] Подполковник Российской императорской армии (1917), участник Первой мировой войны.

[NORMALIZED]
1917 подполковник российский императорский армия участник один мировой война
POS:   5
DOCID:

In [27]:
qid = 1

ranks = ranker.print_serp(queries[qid], relevant_docs[qid], norm=True)
ranks = [(doc_id, int(i < 10)) for i, (doc_id, _) in enumerate(ranks)]
metrics = count_metrics(relevant_docs[qid], ranks)

print()
print(("Precision: {:>.10f}\n" + \
       "Recall:    {:>.10f}\n" + \
       "F-measure: {:>.10f}\n" + \
       "NDCG-10:   {:>.10f}").format(*metrics))

scores[(qid, True)] = metrics

QUERY: Не вызвавший первоначально интереса экспертов «маленький рыжеволосый мальчик» (на илл.) выставлен на аукционе со стартовой ценой в миллион евро.
[NORMALIZED]
не вызвать первоначально интерес эксперт маленький рыжеволосый мальчик на илл выставить на аукцион с стартовый цена в миллион евро


POS:   1
DOCID: 2416
RANK:  0.3828105146

[*] Она оценивается от 3 до 5 млн евро (по другим данным, только в 1 млн евро [2]) — в три раза больше, чем картина «Святой Иероним», которая должна быть выставлена в апреле 2018 года на аукционе «Кристис» (1 млн долларов).

[NORMALIZED]
3 5 1 2 2018 1 она оцениваться от до миллион евро по другой данные только в миллион евро в три раз большой чем картина святой иероним который должный быть выставить в апрель год на аукцион кристис миллион доллар
POS:   2
DOCID: 2389
RANK:  0.2762826438

[*] Владелица увидела Руйяков по телевидению; в разговоре по телефону она называла картину «Маленький рыжеволосый мальчик».

[NORMALIZED]
владелица увидеть руйяк по тел

In [28]:
qid = 2

ranks = ranker.print_serp(queries[qid], relevant_docs[qid], norm=True)
ranks = [(doc_id, int(i < 10)) for i, (doc_id, _) in enumerate(ranks)]
metrics = count_metrics(relevant_docs[qid], ranks)

print()
print(("Precision: {:>.10f}\n" + \
       "Recall:    {:>.10f}\n" + \
       "F-measure: {:>.10f}\n" + \
       "NDCG-10:   {:>.10f}").format(*metrics))

scores[(qid, True)] = metrics

QUERY: Американский фотограф, известный съёмками высшего общества, был также личным фотографом знаменитого мафиозо.
[NORMALIZED]
американский фотограф известный съёмка высокий общество быть также личный фотограф знаменитый мафиозо


POS:   1
DOCID: 2557
RANK:  0.3885780952

[*] Джордж Аллен «Слим» Ааронс (29 октября 1916 — 30 мая 2006) — американский фотограф, известный съемками «красивой жизни» высшего общества, светских персон, кинозвёзд и прочих знаменитостей.

[NORMALIZED]
29 1916 30 2006 джордж аллен слить ааронс октябрь май американский фотограф известный съёмка красивый жизнь высокий общество светский персона кинозвезда и прочий знаменитость
POS:   2
DOCID: 1161
RANK:  0.2889444851

[*] Лаки Лучано был любимой моделью американского фотографа Слима Ааронса.

[NORMALIZED]
лак лучано быть любимый модель американский фотограф слима ааронс
POS:   3
DOCID: 48
RANK:  0.2360169884

[o] Фотография Павла VI, фотограф Лотар Воллех

[NORMALIZED]
фотография павел vi фотограф лотар воллех
POS

### Модель 3: языковая модель

In [29]:
ranker = LangModelRanker(lambda_=0.8)
ranker.load_docs(docs)

In [30]:
qid = 0

ranks = ranker.print_serp(queries[qid], relevant_docs[qid], norm=True, probs=True)
ranks = [(doc_id, int(i < 10)) for i, (doc_id, _) in enumerate(ranks)]
metrics = count_metrics(relevant_docs[qid], ranks)

print()
print(("Precision: {:>.10f}\n" + \
       "Recall:    {:>.10f}\n" + \
       "F-measure: {:>.10f}\n" + \
       "NDCG-10:   {:>.10f}").format(*metrics))

scores[(qid, 2)] = metrics

QUERY: Лётчику понадобилось два десятка лет, чтобы повторно стать подполковником.
[NORMALIZED]
лётчик понадобиться два десятка год чтобы повторно стать подполковник

P(год|model) = 0.01344
P(два|model) = 0.00161
P(стать|model) = 0.00165
P(подполковник|model) = 0.00014
P(лётчик|model) = 0.00037
P(чтобы|model) = 0.00065
P(понадобиться|model) = 0.00001
P(повторно|model) = 0.00009

POS:   1
DOCID: 218
RANK:  1.33530522549381e-27

[o] Категории : Родившиеся 27 ноября Родившиеся в 1890 году Персоналии по алфавиту Родившиеся в Гродненской губернии Кавалеры ордена Ленина Кавалеры ордена Красного Знамени Награждённые медалью «XX лет Рабоче-Крестьянской Красной Армии» Награждённые медалью «За победу над Германией в Великой Отечественной войне 1941—1945 гг.» Кавалеры ордена Святого Георгия IV класса Кавалеры ордена Святой Анны 2 степени Кавалеры ордена Святой Анны 3 степени Кавалеры ордена Святого Станислава 2 степени с мечами Кавалеры ордена Святого Станислава 3 степени Кавалеры Георгиевского ор

In [31]:
qid = 1

ranks = ranker.print_serp(queries[qid], relevant_docs[qid], norm=True, probs=True)
ranks = [(doc_id, int(i < 10)) for i, (doc_id, _) in enumerate(ranks)]
metrics = count_metrics(relevant_docs[qid], ranks)

print()
print(("Precision: {:>.10f}\n" + \
       "Recall:    {:>.10f}\n" + \
       "F-measure: {:>.10f}\n" + \
       "NDCG-10:   {:>.10f}").format(*metrics))

scores[(qid, 2)] = metrics

QUERY: Не вызвавший первоначально интереса экспертов «маленький рыжеволосый мальчик» (на илл.) выставлен на аукционе со стартовой ценой в миллион евро.
[NORMALIZED]
не вызвать первоначально интерес эксперт маленький рыжеволосый мальчик на илл выставить на аукцион с стартовый цена в миллион евро

P(в|model) = 0.03517
P(с|model) = 0.01289
P(на|model) = 0.01069
P(не|model) = 0.00469
P(вызвать|model) = 0.00014
P(миллион|model) = 0.00013
P(маленький|model) = 0.00018
P(интерес|model) = 0.00018
P(первоначально|model) = 0.00013
P(цена|model) = 0.00011
P(эксперт|model) = 0.00013
P(аукцион|model) = 0.00012
P(мальчик|model) = 0.00005
P(выставить|model) = 0.00003
P(рыжеволосый|model) = 0.00002
P(евро|model) = 0.00010

POS:   1
DOCID: 2416
RANK:  9.575889031255278e-53

[*] Она оценивается от 3 до 5 млн евро (по другим данным, только в 1 млн евро [2]) — в три раза больше, чем картина «Святой Иероним», которая должна быть выставлена в апреле 2018 года на аукционе «Кристис» (1 млн долларов).

[NORMALI

In [32]:
qid = 2

ranks = ranker.print_serp(queries[qid], relevant_docs[qid], norm=True, probs=True)
ranks = [(doc_id, int(i < 10)) for i, (doc_id, _) in enumerate(ranks)]
metrics = count_metrics(relevant_docs[qid], ranks)

print()
print(("Precision: {:>.10f}\n" + \
       "Recall:    {:>.10f}\n" + \
       "F-measure: {:>.10f}\n" + \
       "NDCG-10:   {:>.10f}").format(*metrics))

scores[(qid, 2)] = metrics

QUERY: Американский фотограф, известный съёмками высшего общества, был также личным фотографом знаменитого мафиозо.
[NORMALIZED]
американский фотограф известный съёмка высокий общество быть также личный фотограф знаменитый мафиозо

P(быть|model) = 0.00993
P(американский|model) = 0.00038
P(фотограф|model) = 0.00019
P(известный|model) = 0.00059
P(общество|model) = 0.00019
P(также|model) = 0.00156
P(высокий|model) = 0.00052
P(съёмка|model) = 0.00013
P(знаменитый|model) = 0.00011
P(личный|model) = 0.00019

POS:   1
DOCID: 2557
RANK:  7.529517013736699e-25

[*] Джордж Аллен «Слим» Ааронс (29 октября 1916 — 30 мая 2006) — американский фотограф, известный съемками «красивой жизни» высшего общества, светских персон, кинозвёзд и прочих знаменитостей.

[NORMALIZED]
29 1916 30 2006 джордж аллен слить ааронс октябрь май американский фотограф известный съёмка красивый жизнь высокий общество светский персона кинозвезда и прочий знаменитость
POS:   2
DOCID: 1161
RANK:  2.485893136691512e-33

[*] Лаки

## Сравнение моделей

In [33]:
for qid, _ in enumerate(queries):
    print(f"Metrics for QID: {qid+1}\n")
    print("           {:<12} {:<12} {:<12}".format("no IDF", "IDF", "LModel"))
    print("Precision: {:>.10f} {:>.10f} {:>.10f}".format(*(scores[(qid, i)][0] for i in range(3))))
    print("Recall:    {:>.10f} {:>.10f} {:>.10f}".format(*(scores[(qid, i)][1] for i in range(3))))
    print("F-measure: {:>.10f} {:>.10f} {:>.10f}".format(*(scores[(qid, i)][2] for i in range(3))))
    print("NDCG-10:   {:>.10f} {:>.10f} {:>.10f}".format(*(scores[(qid, i)][3] for i in range(3))))
    print("NDCG:      {:>.10f} {:>.10f} {:>.10f}".format(*(scores[(qid, i)][4] for i in range(3))))
    print("=" * 60 + '\n')

Metrics for QID: 1

           no IDF       IDF          LModel      
Precision: 0.1000000000 0.3000000000 0.2000000000
Recall:    0.2000000000 0.6000000000 0.4000000000
F-measure: 0.1333333333 0.4000000000 0.2666666667
NDCG-10:   0.2139862647 0.4468535300 0.3451913422
NDCG:      0.4292651556 0.6227445764 0.5516950142

Metrics for QID: 2

           no IDF       IDF          LModel      
Precision: 0.1000000000 0.2000000000 0.3000000000
Recall:    0.2000000000 0.4000000000 0.6000000000
F-measure: 0.1333333333 0.2666666667 0.4000000000
NDCG-10:   0.3937481556 0.6421755823 0.7559943246
NDCG:      0.5711782645 0.7827567770 0.7988582470

Metrics for QID: 3

           no IDF       IDF          LModel      
Precision: 0.2000000000 0.2000000000 0.2000000000
Recall:    0.5000000000 0.5000000000 0.5000000000
F-measure: 0.2857142857 0.2857142857 0.2857142857
NDCG-10:   0.5606626602 0.5606626602 0.5606626602
NDCG:      0.6494015612 0.6681382680 0.6692960993



In [34]:
print(f"Mean metrics for QIDs\n")
print("{:<10s} {:<12} {:<12} {:<12}".format("", "no IDF", "IDF", "LModel"))

for i, metric in enumerate(["Precision:", "Recall:", "F-measure:", "NDCG-10:", "NDCG:"]):
    score_0 = np.mean([scores[(qid, 0)][i] for qid in range(len(queries))])
    score_1 = np.mean([scores[(qid, 1)][i] for qid in range(len(queries))])
    score_2 = np.mean([scores[(qid, 2)][i] for qid in range(len(queries))])
    
    print("{:<10s} {:>.10f} {:>.10f} {:>.10f}".format(metric, score_0, score_1, score_2))

Mean metrics for QIDs

           no IDF       IDF          LModel      
Precision: 0.1333333333 0.2333333333 0.2333333333
Recall:    0.3000000000 0.5000000000 0.5000000000
F-measure: 0.1841269841 0.3174603175 0.3174603175
NDCG-10:   0.3894656935 0.5498972575 0.5539494423
NDCG:      0.5499483271 0.6912132071 0.6732831202
